In [ ]:
import MeCab
m = MeCab.Tagger('-d /usr/lib/mecab/dic/mecab-ipadic-neologd')

In [ ]:
import pandas as pd

csv_df = pd.read_csv('example.csv',encoding="utf8") 
len(csv_df)
csv_df.info()
csv_df.head()

In [ ]:
import jaconv
wakatis = []
words = []
for w in csv_df['歳出事業']:
    words1 = []
    w = jaconv.z2h(w, digit=True, ascii=True) # 半角、小文字に統一
    for c in m.parse(w).splitlines()[:-1]:  #事業名を分割し解析結果を配列にして繰り返す
        surface, feature = c.split('\t')    #分割された事業名と結果をそれぞれ格納 
        if '名詞' in feature:             #名詞ならそれをwords1に追加
            if surface != '推進事業' and surface != '事業':
                if surface == '健康診査':
                    words1.append('健康')
                    words1.append('診査')
                elif surface == '庄内地域':
                    words1.append('庄内')
                    words1.append('地域')
                else:
                    words1.append(surface)
            
                
    str1 = ' '.join(words1) #分割された事業名を空白で区切る
    wakatis.append(str1)    #区切られた事業名を配列に格納
    words.append(words1)
# wakatis
csv_df['wakati'] = wakatis
learn_df = csv_df[['歳出事業コード','歳出事業','wakati']]
learn_df.info()
learn_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
transformer = TfidfTransformer()

tf = vectorizer.fit_transform(wakatis) # 単語の出現頻度を計算
tfidf = transformer.fit_transform(tf)

while True:
    
    word = input('検索:')
    if word == '終了':
        break
    
    else:
        t1 = time.time()
        words2 = []
        for c in m.parse(word).splitlines()[:-1]:
            surface, feature = c.split('\t')    #分割された事業名と結果をそれぞれ格納 
            if '名詞' in feature:
                words2.append(surface)
        str2 = ' '.join(words2) #分割された事業名を空白で区切る
        wakatis.append(str2)
        sample_tf = vectorizer.transform([wakatis[-1]])
        sample_tfidf = transformer.transform(sample_tf)
        similarity = cosine_similarity(sample_tfidf, tfidf)[0]
        topn_indices = np.argsort(similarity)[::-1][0:10]
        b = True
        for sim, tweet in zip(similarity[topn_indices], np.array(csv_df['歳出事業'])[topn_indices]):
            if sim != 0:
                print("({:}): {}".format(sim, "".join(tweet.split())))
                b = False
            if b:
                print('検索結果なし')
                break
        del wakatis[-1]
        print('検索時間:', time.time()-t1)